In [8]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [9]:
import requests
requests.get("http://localhost:11434").content

b'Ollama is running'

In [10]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
from openai import OpenAI
openai = OpenAI()
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [11]:
# Get a fun fact
response = ollama.chat.completions.create(model="llama3.2", messages=[{"role":"user","content":"Tell me a fun fact"}])
response.choices[0].message.content

'Here\'s one:\n\nDid you know that there is a species of jellyfish that is immortal? The Turritopsis dohrnii, also known as the "immortal jellyfish," can transform its body into a younger state through a process called transdifferentiation. This means that it can essentially revert back to its polyp stage and grow back into an adult again, making it theoretically immortal!\n\nIsn\'t that just mind-blowing?'

In [12]:
from scraper import fetch_website_contents
from IPython.display import Markdown, display

In [14]:
# A simple QnA
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = ollama.chat.completions.create(model="llama3.2", messages=messages)
response.choices[0].message.content

'The answer to 2 + 2 is 4.'

In [ ]:
# Fetching website contents
ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

In [15]:
# Define our system prompt 
system_prompt = """
You are a snarky assistant that analyzes the contents of website,
and provides a short, snarky humorous summary, ignoring text that might be navigation related.
Respons in markdown. Do not wrap the markdown in a code block - respond just with the markdown."""

In [ ]:
# Define our user prompt
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or annoucements, then summarize these too. 
"""

In [17]:
def messages_for(website):
    return[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [18]:
messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of website,\nand provides a short, snarky humorous summary, ignoring text that might be navigation related.\nRespons in markdown. Do not wrap the markdown in a code block - respond just with the markdown.'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or annoucements, then summarize these too. \nHome - Edward Donner\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re 

In [19]:
# Summarize the website contents
def summarize(url):
    website = fetch_website_contents(url)
    response = ollama.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [20]:
summarize("https://edwarddonner.com")

"A self-promotional website from a tech enthusiast and AI CTO. Ed's site is all about his work at Nebula.io, where they're disrupting recruitment by leveraging LLMs to find the perfect talent match.\n\n**No recent news or announcements mentioned, but some upcoming events:**\n\n* The Unique Energy of an AI Live Event (Nov 11)\n* AI in Production: Gen AI and Agentic AI on AWS at scale (Sep 15)\n* Be an AI Engineer and Leader: The Curriculum (May 18)\n* 2025 AI Executive Briefing\n\nNot much snark to be had here, folks!"

In [21]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [22]:
display_summary("https://edwarddonner.com")

# Summary of the Website
The website appears to be the personal blog of Edward Donner, the co-founder and CTO of Nebula.io. He's a self-proclaimed nerd with interests in coding, AI, DJing, and amateur music production.

Edward discusses his career as an entrepreneur and tech enthusiast, including his time as the founder and CEO of an acquired AI startup, untapt. He is passionate about applying AI to help people discover their potential and has developed an award-winning platform for matching talent with employers.

### News/Announcements

* **The Unique Energy of an AI Live Event**: Scheduled for November 11, 2025
* **AI in Production: Gen AI and Agentic AI on AWS at scale**: Mentioned on September 15, 2025 (exact details not provided)
* **Be an AI Engineer and Leader: The Curriculum**
* **2025 AI Executive Briefing**

In [23]:
display_summary("https://cnn.com")

**Summary:** A snarky take on CNN.com - it's like your grandma who won't stop talking, but with less judgmental looks.

**News and Announcements:**

* Breaking News: Ukraine, don't get caught up in this war thing. We'll try not to editorialize too much. 
* Latest News: Stay informed about the latest developments in Israel-Hamas War, 2025 Elections, Redistricting Tracker... you know, the usual suspects.

**Other Observations:** There's an ad issue; it loaded, but not without some hiccups (pun intended). Other users are brave enough to share their thoughts on how relevant ads are and what issues they encountered. Bless them for trying to provide feedback!

In [24]:
display_summary("https://anthropic.com")

**Anthropic: Saving Humanity from AI's Looming Existence**

This website is all about saving humanity from itself through Artificial Intelligence. Founded as a public benefit corporation, Anthropic aims to harness AI's power for good while mitigating its risks.

**News Updates**

* Introducing Claude Sonnet 4.5, the "best model in the world" for agents, coding, and computer use ( announced multiple times on the site, because why not try again?)
* Meet Claude Haiku 4.5, a game-changer for managing context on the Claude Developer Platform
* What is it like to talk to a large language model? Apparently, it's about figuring out who you're actually having a conversation with.

**Other Interesting Bits**

The site boasts of building AI to serve humanity's long-term well-being. Um, what does that even mean?

Overall, Anthropic seems determined to take on the world's most pressing question: "AI will have a vast impact on the world."